# Short Straddle Intraday Without Take Profit

In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# === CONFIG ===
ENTRY_WINDOW_START = "09:20:00"  # Start of entry window for finding optimal entry
ENTRY_WINDOW_END = "09:45:00"    # End of entry window for finding optimal entry
EXIT_TIME = "15:15:00"
SLIPPAGE = 0.005  # per leg
LOTSIZE = 375
BROKERAGE_PER_LEG = 0.5  # per leg per lot
SL_PCT = 0.1
INITIAL_CAPITAL = 1000000
MAX_REENTRIES = 3  # Maximum number of re-entries after stop loss
DATA_FOLDER = r"C:\Users\omdes\Desktop\QuantInsti\AlphaNiti Assignment\Options data\APR_2021"

def load_data(folder_path):
    """
    Load and combine all CSV files from the specified folder
    """
    all_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)
                # Combine Date + Time into timestamp
                df['timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
                # Standardize column names
                df.columns = df.columns.str.lower().str.replace(" ", "_")
                all_data.append(df)
            except Exception as e:
                print(f"Error loading {file}: {e}")
    
    if all_data:
        df_all = pd.concat(all_data).reset_index(drop=True)
        print("Combined Data Shape:", df_all.shape)
        print(df_all[['ticker', 'timestamp', 'open', 'high', 'low', 'close']].head())
        return df_all
    else:
        print("No data loaded!")
        return None

def get_atm_strike_and_prices(df_day, timestamp=None):
    """
    Identify the ATM strike and return its CE/PE tickers
    
    Parameters:
    df_day - DataFrame with day's data
    timestamp - Optional specific timestamp to use (for re-entries)
    """
    if timestamp:
        # For re-entries, use the specified timestamp with a 2-minute window
        ts_time = pd.to_datetime(timestamp)
        start_time = (ts_time - timedelta(minutes=1)).strftime('%H:%M:%S')
        end_time = (ts_time + timedelta(minutes=1)).strftime('%H:%M:%S')
        window_df = df_day[df_day['timestamp'].dt.strftime('%H:%M:%S').between(start_time, end_time)]
    else:
        # For initial entry, use the specified entry window
        window_df = df_day[df_day['timestamp'].dt.strftime('%H:%M:%S').between(ENTRY_WINDOW_START, ENTRY_WINDOW_END)]
    
    window_df = window_df[window_df['ticker'].str.startswith("NIFTY")].copy()

    if window_df.empty:
        return None, None, None, None

    window_df['strike'] = window_df['ticker'].str.extract(r'NIFTY\d{2}[A-Z]{3}\d{2}(\d{4,5})').astype(float)
    window_df['option_type'] = window_df['ticker'].str.extract(r'(CE|PE)')

    # Group by timestamp and strike to find ATM strike at each timestamp
    atm_candidates = []
    for ts, group in window_df.groupby('timestamp'):
        pivot = group.pivot_table(index='strike', columns='option_type', values='close', aggfunc='mean')
        if 'CE' in pivot.columns and 'PE' in pivot.columns:
            pivot = pivot.dropna()
            pivot['diff'] = abs(pivot['CE'] - pivot['PE'])
            
            if not pivot.empty:
                atm_strike = pivot['diff'].idxmin()
                ce_price = pivot.loc[atm_strike, 'CE']
                pe_price = pivot.loc[atm_strike, 'PE']
                premium_sum = ce_price + pe_price
                
                atm_candidates.append({
                    'timestamp': ts,
                    'strike': atm_strike,
                    'ce_price': ce_price,
                    'pe_price': pe_price,
                    'premium_sum': premium_sum
                })
    
    if not atm_candidates:
        return None, None, None, None
    
    # Sort by premium sum (highest premium = best initial entry for short straddle)
    atm_candidates_df = pd.DataFrame(atm_candidates)
    best_entry = atm_candidates_df.sort_values('premium_sum', ascending=False).iloc[0]
    
    optimal_ts = best_entry['timestamp']
    atm_strike = best_entry['strike']
    
    # Extract tickers for optimal strike
    ts_time = optimal_ts.strftime('%H:%M:%S')
    ts_window_df = window_df[window_df['timestamp'].dt.strftime('%H:%M:%S') == ts_time]
    
    ce_ticker = ts_window_df[(ts_window_df['strike'] == atm_strike) & 
                        (ts_window_df['option_type'] == 'CE')]['ticker'].iloc[0]
    pe_ticker = ts_window_df[(ts_window_df['strike'] == atm_strike) & 
                        (ts_window_df['option_type'] == 'PE')]['ticker'].iloc[0]

    entry_time = optimal_ts if not timestamp else timestamp
    print(f"\nBest Entry Found at: {entry_time.strftime('%H:%M:%S')}")
    print(f"ATM Strike: {atm_strike}")
    print(f"CE Ticker: {ce_ticker}, Premium: {best_entry['ce_price']}")
    print(f"PE Ticker: {pe_ticker}, Premium: {best_entry['pe_price']}")
    print(f"Total Premium: {best_entry['premium_sum']}")

    return atm_strike, ce_ticker, pe_ticker, optimal_ts

def backtest_short_straddle(df_all, date):
    """
    Backtest short straddle strategy for a specific date with re-entries
    """
    print(f"\n=== Backtesting for {date} ===")
    df_day = df_all[df_all['timestamp'].dt.date == pd.to_datetime(date).date()]
    df_day = df_day[df_day['ticker'].str.startswith("NIFTY")]

    if df_day.empty:
        print(f"No data available for {date}")
        return None

    # List to store all trades for this day
    all_trades = []
    
    # Track the number of entries made today
    entry_count = 0
    
    # Get initial tickers with optimal entry time
    atm_strike, ce_ticker, pe_ticker, optimal_entry_time = get_atm_strike_and_prices(df_day)
    if not atm_strike:
        print(f"Could not determine ATM strike for {date}")
        return None
    
    # Extract all timestamps for the day
    all_timestamps = sorted(df_day['timestamp'].unique())
    
    # Initialize for first entry using optimal time
    next_entry_time = optimal_entry_time
    
    # Continue trading until we hit max entries or market close
    while entry_count <= MAX_REENTRIES and next_entry_time.strftime('%H:%M:%S') <= EXIT_TIME:
        # Check if this is a re-entry
        is_reentry = entry_count > 0
        
        if is_reentry:
            # For re-entries, get new ATM strikes based on current market conditions
            atm_strike, ce_ticker, pe_ticker, _ = get_atm_strike_and_prices(df_day, next_entry_time)
            if not atm_strike:
                print(f"Could not determine ATM strike for re-entry at {next_entry_time}")
                break
        
        # Get data for selected tickers
        ce_df = df_day[df_day['ticker'] == ce_ticker]
        pe_df = df_day[df_day['ticker'] == pe_ticker]
        
        # Find entry prices
        ce_entry_row = ce_df[ce_df['timestamp'] == next_entry_time]
        pe_entry_row = pe_df[pe_df['timestamp'] == next_entry_time]
        
        if ce_entry_row.empty or pe_entry_row.empty:
            print(f"Entry data not available at {next_entry_time} for {date}")
            break
        
        # Entry prices
        ce_entry = ce_entry_row['close'].iloc[0]
        pe_entry = pe_entry_row['close'].iloc[0]
        gross_entry = ce_entry + pe_entry
        
        # Calculate SL level
        sl_premium = gross_entry * (1 + SL_PCT)
        
        entry_time_str = next_entry_time.strftime('%H:%M:%S')
        entry_label = "REENTRY" if is_reentry else "INITIAL"
        
        print(f"\n--- {entry_label} ENTRY #{entry_count+1} at {entry_time_str} ---")
        print(f"CE Entry (SELL): {ce_entry}")
        print(f"PE Entry (SELL): {pe_entry}")
        print(f"Gross Premium Received: {gross_entry}")
        print(f"Stop Loss Premium Level: {sl_premium} (Risk: {SL_PCT*100}%)")
        
        # Track this trade
        merged = pd.merge(
            ce_df[['timestamp', 'close']],
            pe_df[['timestamp', 'close']],
            on='timestamp',
            suffixes=('_ce', '_pe')
        ).sort_values('timestamp')
        
        # Only consider data after this entry time
        merged = merged[merged['timestamp'] >= next_entry_time]
        
        # Trade management variables
        active_trade = True
        next_entry_time = None  # Will be set if SL is hit and we need to re-enter
        
        for _, row in merged.iterrows():
            ts = row['timestamp']
            if ts.strftime('%H:%M:%S') > EXIT_TIME:
                break
                
            curr_premium = row['close_ce'] + row['close_pe']
            
            # Check for stop loss hit
            if active_trade and curr_premium >= sl_premium:
                # Calculate PnL for a short position: (entry - exit) * lotsize - brokerage
                pnl = (gross_entry - curr_premium - (2 * BROKERAGE_PER_LEG)) * LOTSIZE
                all_trades.append({
                    'date': date,
                    'entry_number': entry_count + 1,
                    'entry_type': entry_label,
                    'entry_time': next_entry_time,
                    'exit_time': ts,
                    'ce_entry': ce_entry,
                    'pe_entry': pe_entry,
                    'ce_exit': row['close_ce'],
                    'pe_exit': row['close_pe'],
                    'exit_reason': 'SL_HIT',
                    'PnL': pnl
                })
                active_trade = False
                print(f"SL hit at {ts.strftime('%H:%M:%S')} - PnL: {pnl}")
                
                # Set up for re-entry if we haven't hit the maximum
                if entry_count < MAX_REENTRIES:
                    next_entry_time = ts  # Re-enter at the time of SL hit
                    entry_count += 1
                break
        
        # If we made it to market close without hitting SL
        if active_trade:
            last = merged[merged['timestamp'].dt.strftime('%H:%M:%S') <= EXIT_TIME].iloc[-1]
            curr_premium = last['close_ce'] + last['close_pe']
            pnl = (gross_entry - curr_premium - (2 * BROKERAGE_PER_LEG)) * LOTSIZE
            all_trades.append({
                'date': date,
                'entry_number': entry_count + 1,
                'entry_type': entry_label,
                'entry_time': next_entry_time,
                'exit_time': last['timestamp'],
                'ce_entry': ce_entry,
                'pe_entry': pe_entry,
                'ce_exit': last['close_ce'],
                'pe_exit': last['close_pe'],
                'exit_reason': 'MARKET_CLOSE',
                'PnL': pnl
            })
            print(f"Position closed at market close - PnL: {pnl}")
            # No more trading after market close
            break
        
        # If no next entry time was set (other reason), break the loop
        if next_entry_time is None:
            break
            
        # Move to the next entry
        entry_count += 1 if entry_count == 0 else 0  # Only increment if this was the first entry
    
    if all_trades:
        return pd.DataFrame(all_trades)
    else:
        print(f"No trades executed for {date}")
        return None

def generate_performance_report(results_df):
    """
    Generate performance metrics from backtest results
    """
    results_df['date'] = pd.to_datetime(results_df['date'])
    results_df = results_df.sort_values(['date', 'entry_time'])

    # Add daily statistics
    daily_stats = []
    for date, group in results_df.groupby('date'):
        daily_pnl = group['PnL'].sum()
        daily_stats.append({
            'date': date,
            'trades': len(group),
            'daily_pnl': daily_pnl
        })
    
    daily_df = pd.DataFrame(daily_stats)
    daily_df['cum_pnl'] = daily_df['daily_pnl'].cumsum()
    daily_df['equity'] = INITIAL_CAPITAL + daily_df['cum_pnl']
    daily_df['daily_return'] = daily_df['daily_pnl'] / daily_df['equity'].shift(1)
    daily_df['daily_return'] = daily_df['daily_return'].fillna(0)

    # === Performance Metrics ===
    total_days = len(daily_df)
    profitable_days = sum(daily_df['daily_pnl'] > 0)
    win_rate_days = profitable_days / total_days if total_days > 0 else 0
    
    total_trades = len(results_df)
    profitable_trades = sum(results_df['PnL'] > 0)
    win_rate_trades = profitable_trades / total_trades if total_trades > 0 else 0
    
    # Segregate by exit reason
    sl_hits = sum(results_df['exit_reason'] == 'SL_HIT')
    market_closes = sum(results_df['exit_reason'] == 'MARKET_CLOSE')
    
    # Re-entry statistics
    entry_counts = results_df.groupby('date')['entry_number'].max().value_counts().sort_index()
    avg_entries_per_day = results_df.groupby('date')['entry_number'].max().mean()
    
    total_return = daily_df['equity'].iloc[-1] - INITIAL_CAPITAL
    percentage_return = (total_return / INITIAL_CAPITAL) * 100
    
    max_drawdown = (daily_df['equity'].cummax() - daily_df['equity']).max()
    max_drawdown_pct = max_drawdown / daily_df['equity'].cummax().max() * 100
    
    # Annualized metrics (assuming 252 trading days in a year)
    days_elapsed = (daily_df['date'].iloc[-1] - daily_df['date'].iloc[0]).days
    years = days_elapsed / 365
    cagr = ((daily_df['equity'].iloc[-1] / INITIAL_CAPITAL) ** (1 / years if years > 0 else 1)) - 1
    sharpe = daily_df['daily_return'].mean() / daily_df['daily_return'].std() * np.sqrt(252) if daily_df['daily_return'].std() > 0 else 0

    print("\n==== Performance Summary ====")
    print(f"Total Days     : {total_days}")
    print(f"Total Trades   : {total_trades}")
    print(f"Avg Trades/Day : {total_trades / total_days:.2f}")
    print(f"Avg Entries/Day: {avg_entries_per_day:.2f}")
    print(f"Win Rate (Days): {win_rate_days:.2%}")
    print(f"Win Rate (Trades): {win_rate_trades:.2%}")
    print(f"Total Return   : ₹{total_return:.2f} ({percentage_return:.2f}%)")
    print(f"Max Drawdown   : ₹{max_drawdown:.2f} ({max_drawdown_pct:.2f}%)")
    print(f"CAGR           : {cagr * 100:.2f}%")
    print(f"Sharpe Ratio   : {sharpe:.2f}")
    
    print("\n==== Entry Statistics ====")
    for entry_num, count in entry_counts.items():
        print(f"{entry_num} Entry(ies): {count} days ({count/total_days:.2%} of days)")
    
    print("\n==== Exit Statistics ====")
    print(f"Stop Loss Hits : {sl_hits} ({sl_hits/total_trades:.2%} of trades)")
    print(f"Market Close   : {market_closes} ({market_closes/total_trades:.2%} of trades)")

    # === Daily Table ===
    summary_table = daily_df[['date', 'trades', 'daily_pnl', 'equity']].copy()
    summary_table.columns = ['Date', 'Trades', 'Daily PnL', 'Capital']
    summary_table['Date'] = summary_table['Date'].dt.strftime('%Y-%m-%d')
    summary_table['Daily PnL'] = summary_table['Daily PnL'].map(lambda x: f"{x:+,.2f}")
    summary_table['Capital'] = summary_table['Capital'].map(lambda x: f"{x:,.2f}")

    print("\n==== Daily Capital & PnL ====")
    print(summary_table.to_string(index=False))

    # Return combined daily and trade data
    return {
        'daily': daily_df,
        'trades': results_df
    }

def run_backtest(start_date, end_date):
    """
    Run the backtest for a specified date range
    """
    print(f"==== Running Short Straddle Backtest with Re-entries ====")
    print(f"Date Range: {start_date} to {end_date}")
    print(f"Entry Window: {ENTRY_WINDOW_START} to {ENTRY_WINDOW_END} (finding optimal entry)")
    print(f"Exit Time: {EXIT_TIME}")
    print(f"Lot Size: {LOTSIZE}")
    print(f"Stop Loss: {SL_PCT*100}% of entry premium")
    print(f"Max Re-entries: {MAX_REENTRIES}")
    print(f"Initial Capital: ₹{INITIAL_CAPITAL:,}")
    
    # Load data
    df_all = load_data(DATA_FOLDER)
    if df_all is None:
        return
    
    # Get all trading days in the specified range
    all_dates = pd.date_range(start=start_date, end=end_date, freq='B')
    all_trades = []

    for d in all_dates:
        day_result = backtest_short_straddle(df_all, d.strftime('%Y-%m-%d'))
        if day_result is not None:
            all_trades.append(day_result)

    if all_trades:
        results = pd.concat(all_trades)
        results_data = generate_performance_report(results)
        
        # Export results to CSV
        trade_filename = f"ShortStraddle_Trades_{start_date}_to_{end_date}.csv"
        daily_filename = f"ShortStraddle_Daily_{start_date}_to_{end_date}.csv"
        
        results_data['trades'].to_csv(trade_filename, index=False)
        results_data['daily'].to_csv(daily_filename, index=False)
        
        print(f"\nTrade results exported to {trade_filename}")
        print(f"Daily results exported to {daily_filename}")
        
        return results_data
    else:
        print("No trades executed during the backtest period.")
        return None

# === MAIN EXECUTION ===
if __name__ == "__main__":
    start_date = '2021-04-01'
    end_date = '2021-04-30'
    
    results_data = run_backtest(start_date, end_date)
    
    if results_data is not None:
        # You can add code here to create plots or additional analysis
        print("\nBacktest completed successfully!")
    else:
        print("\nBacktest failed to execute properly.")

==== Running Short Straddle Backtest with Re-entries ====
Date Range: 2021-04-01 to 2021-04-30
Entry Window: 09:20:00 to 09:45:00 (finding optimal entry)
Exit Time: 15:15:00
Lot Size: 375
Stop Loss: 10.0% of entry premium
Max Re-entries: 3
Initial Capital: ₹1,000,000
Combined Data Shape: (12459881, 10)
                      ticker           timestamp  open  high   low  close
0  AARTIIND29APR211100PE.NFO 2021-04-01 14:16:59   2.2   2.2  2.20   2.20
1  AARTIIND29APR211200PE.NFO 2021-04-01 10:16:59   8.5   8.5  8.50   8.50
2  AARTIIND29APR211200PE.NFO 2021-04-01 11:43:59   8.5   8.5  8.50   8.50
3  AARTIIND29APR211200PE.NFO 2021-04-01 12:36:59   8.5   8.5  6.05   6.05
4  AARTIIND29APR211200PE.NFO 2021-04-01 12:37:59   6.1   6.1  6.10   6.10

=== Backtesting for 2021-04-01 ===

Best Entry Found at: 09:43:59
ATM Strike: 15000.0
CE Ticker: NIFTY01APR2115000CE.NFO, Premium: 315.7416666666667
PE Ticker: NIFTY01APR2115000PE.NFO, Premium: 351.65
Total Premium: 667.3916666666667

--- INITIAL ENTR

## ==== Performance Summary ====
- Total Days     : 19
- Total Trades   : 28
- Avg Trades/Day : 1.47
- Avg Entries/Day: 1.47
- Win Rate (Days): 52.63%
- Win Rate (Trades): 46.43%
- Total Return   : ₹27056.25 (2.71%)
- Max Drawdown   : ₹31575.00 (2.98%)
- CAGR           : 39.94%
- Sharpe Ratio   : 1.58

## ==== Entry Statistics ====
- 1 Entry(ies): 14 days (73.68% of days)
- 2 Entry(ies): 2 days (10.53% of days)
- 3 Entry(ies): 2 days (10.53% of days)
- 4 Entry(ies): 1 days (5.26% of days)

## ==== Exit Statistics ====
- Stop Loss Hits : 9 (32.14% of trades)
- Market Close   : 19 (67.86% of trades)

## ==== Daily Capital & PnL ====

- Trade results exported to ShortStraddle_Trades_2021-04-01_to_2021-04-30.csv
- Daily results exported to ShortStraddle_Daily_2021-04-01_to_2021-04-30.csv

- Backtest completed successfully!